In [ ]:
%pip install numpy==1.24.3 opencv-python==4.8.0.76 --force-reinstall

In [ ]:
import os

# File to delete
file_path = "/kaggle/working/yolo11n.pt"

# Check if it exists and remove it
if os.path.exists(file_path) and os.path.isfile(file_path):
    os.remove(file_path)
    print(f"Deleted file: {file_path}")
else:
    print(f"File does not exist: {file_path}")


In [ ]:
import shutil
import os

# Source folder (your uploaded dataset)
src_folder = "/kaggle/input/yolo-dataset/All-weapons-data-1"

# Destination folder in working directory
dst_folder = "/kaggle/working/All-weapons-data-1"

# Copy entire folder and overwrite if it exists
shutil.copytree(src_folder, dst_folder, dirs_exist_ok=True)

print(f"Dataset copied to: {dst_folder}")


In [ ]:
import yaml

yaml_path = "/kaggle/working/All-weapons-data-1/data.yaml"

# Load YAML
with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

# Update paths to point to 'images' folders
data['train'] = "/kaggle/working/All-weapons-data-1/train/images"
data['val']   = "/kaggle/working/All-weapons-data-1/valid/images"
data['test']  = "/kaggle/working/All-weapons-data-1/test/images"

# Save updated YAML
with open(yaml_path, 'w') as f:
    yaml.safe_dump(data, f)

print("Updated data.yaml paths to point directly to images folders.")


In [ ]:
%pip install ultralytics

In [ ]:
# Shell command to copy the file to the writable working directory
!cp /kaggle/input/weapon-model/pytorch/default/1/last.pt /kaggle/working/

In [ ]:
import pandas as pd
import numpy as np
import os
from collections import Counter
import ultralytics

In [ ]:

# from roboflow import Roboflow
# rf = Roboflow(api_key="iqWudABIjfXqATDPAmLo")
# project = rf.workspace("custome-yolo").project("all-weapons-data-jqimy")
# version = project.version(1)
# dataset = version.download("yolov8",location="C:/Users/PMLS/Downloads")


We will first check if all the train , test and valid set have correct corresponding images and labels or not


We dont have to worry about dimensions , yolo will take care of it automatically.

In [ ]:
# import os
# from collections import Counter

# label_dir = "D:/ML/Tasks/CNN/All-weapons-data-1/train/labels"  # adjust path if needed
# all_labels = []

# for file in os.listdir(label_dir):
#     if file.endswith(".txt"):
#         with open(os.path.join(label_dir, file)) as f:
#             for line in f:
#                 class_id = int(line.split()[0])
#                 all_labels.append(class_id)




In [ ]:
# print(Counter(all_labels))

In [ ]:
from ultralytics import YOLO

In [ ]:

model = YOLO("yolov8n.yaml")  # just the architecture, no weights
model.train(
    data="/kaggle/working/All-weapons-data-1/data.yaml",
    project="/kaggle/working",
    name="weapon_model",
    epochs=100,
    imgsz=640,
    batch=16,
    lr0=0.001,
    lrf=0.01,
    amp=False,
    optimizer="SGD",
    momentum=0.937,
    weight_decay=0.0005,
    workers=0,
    augment=True,
    mosaic=True,
    mixup=True,
    flipud=0.5,
    fliplr=0.5,
    degrees=15,
    translate=0.1,
    scale=0.1
)



In [ ]:
import os
import shutil
from pathlib import Path
from ultralytics import YOLO

# Configuration
CHECKPOINT_PATH = "/kaggle/working/last.pt"
DATA_YAML = "/kaggle/working/All-weapons-data-1/data.yaml"
PROJECT_DIR = "/kaggle/working"
MODEL_NAME = "weapon_model_resume"
TOTAL_EPOCHS = 100  # Total epochs you want to train
EPOCHS_PER_ITERATION = 10  # Train 10 epochs at a time

# Calculate number of iterations needed
num_iterations = TOTAL_EPOCHS // EPOCHS_PER_ITERATION

print(f"Starting training loop: {num_iterations} iterations of {EPOCHS_PER_ITERATION} epochs each")
print(f"Total target epochs: {TOTAL_EPOCHS}")
print("=" * 60)

for iteration in range(num_iterations):
    print(f"\n{'='*60}")
    print(f"ITERATION {iteration + 1}/{num_iterations}")
    print(f"{'='*60}\n")
    
    try:
        # Load the model from checkpoint
        if os.path.exists(CHECKPOINT_PATH):
            print(f"Loading checkpoint from: {CHECKPOINT_PATH}")
            model = YOLO(CHECKPOINT_PATH)
        else:
            print(f"Checkpoint not found at {CHECKPOINT_PATH}")
            print("Starting fresh training (this should only happen on first run)")
            model = YOLO("yolov8n.yaml")  # Replace with your base model if different
        
        # Train for EPOCHS_PER_ITERATION epochs
        print(f"\nStarting training for {EPOCHS_PER_ITERATION} epochs...")
        results = model.train(
            data=DATA_YAML,
            project=PROJECT_DIR,
            name=MODEL_NAME,
            epochs=EPOCHS_PER_ITERATION,
            resume=False,  # Set to False since we're managing epochs manually
            imgsz=640,
            batch=16,
            lr0=0.001,
            lrf=0.01,
            amp=False,
            optimizer="SGD",
            momentum=0.937,
            weight_decay=0.0005,
            workers=0,
            augment=True,
            mosaic=True,
            mixup=True,
            flipud=0.5,
            fliplr=0.5,
            degrees=15,
            translate=0.1,
            scale=0.1
        )
        
        # Find the newly trained model (last.pt from the run directory)
        run_dir = Path(PROJECT_DIR) / MODEL_NAME
        new_checkpoint = run_dir / "weights" / "last.pt"
        
        if new_checkpoint.exists():
            print(f"\nTraining iteration completed!")
            print(f"Copying checkpoint from: {new_checkpoint}")
            print(f"                     to: {CHECKPOINT_PATH}")
            
            # Copy the new checkpoint to our main checkpoint location
            shutil.copy2(new_checkpoint, CHECKPOINT_PATH)
            print("Checkpoint updated successfully!")
            
            # Optional: Download the checkpoint (Kaggle specific)
            # This creates a copy in the output directory which Kaggle can download
            try:
                output_checkpoint = f"/kaggle/working/checkpoint_iter_{iteration + 1}.pt"
                shutil.copy2(CHECKPOINT_PATH, output_checkpoint)
                print(f"Checkpoint copied to output: {output_checkpoint}")
                print("  (This file will be available for download after session ends)")
            except Exception as e:
                print(f"Warning: Could not create downloadable copy: {e}")
            
            # Also copy best.pt if available
            best_checkpoint = run_dir / "weights" / "best.pt"
            if best_checkpoint.exists():
                best_output = f"/kaggle/working/best_iter_{iteration + 1}.pt"
                shutil.copy2(best_checkpoint, best_output)
                print(f"Best model copied to: {best_output}")
        else:
            print(f"Warning: Could not find checkpoint at {new_checkpoint}")
            print("Training may have failed or checkpoint location changed.")
        
        print(f"\nCompleted iteration {iteration + 1}/{num_iterations}")
        print(f"Total epochs trained so far: {(iteration + 1) * EPOCHS_PER_ITERATION}")
        
    except Exception as e:
        print(f"\n Error during iteration {iteration + 1}: {str(e)}")
        print("Attempting to continue with next iteration...")
        continue

print("\n" + "=" * 60)
print("TRAINING LOOP COMPLETED!")
print("=" * 60)
print(f"Final checkpoint available at: {CHECKPOINT_PATH}")
print("\nAll iteration checkpoints are saved in /kaggle/working/ for download.")